# Neural Networks using TensorFlow

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import src.mnist
import src.const

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
%matplotlib inline

In [37]:
train_set, test_set = src.mnist.get_mnist_full(validation=False)

dtype: <class 'numpy.float64'>
dtype: <class 'numpy.float64'>


In [38]:
def reformat(dataset):
    labels = (np.arange(10) == dataset.labels[:,None]).astype(np.float64)
    return src.const.set_mnist(dataset, 'labels', labels)

train_set = reformat(train_set)
test_set = reformat(test_set)

In [25]:
learning_rate = 0.001
training_epochs = 30
batch_size = 100
display_step = 1

n_hidden_1 = 1024
n_hidden_2 = 512
n_hidden_3 = 256
n_hidden_4 = 128
n_input = 784
n_classes = 10

graph = tf.Graph()
with graph.as_default():
    
    x = tf.placeholder("float", [None, n_input])
    y = tf.placeholder("float", [None, n_classes])
    global_step = tf.Variable(0)

    def multilayer_perceptron(x, weights, biases):

        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        layer_1 = tf.nn.relu(layer_1)

        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        layer_2 = tf.nn.relu(layer_2)
        
        layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
        layer_3 = tf.nn.relu(layer_3)

        layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
        layer_4 = tf.nn.relu(layer_4)
        
        out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
        return out_layer

    weights = {
        'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
        'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
        'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4])),
        'out': tf.Variable(tf.random_normal([n_hidden_4, n_classes]))
    }

    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'b3': tf.Variable(tf.random_normal([n_hidden_3])),
        'b4': tf.Variable(tf.random_normal([n_hidden_4])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }

    pred = multilayer_perceptron(x, weights, biases)

    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    
    exp_lr = tf.train.exponential_decay(learning_rate, global_step, 1000, 0.90, staircase=True)
    optimizer = tf.train.AdamOptimizer(exp_lr).minimize(cost, global_step=global_step)


In [39]:
def run_session(num_steps=1001):
    with tf.Session(graph=graph) as sess:
        tf.initialize_all_variables().run()

        print("Initialized")
        prev, curr = -1, 0
        epoch = 0

        while abs(curr - prev) > 0.1:
            prev = curr

            avg_cost = 0.
            total_batch = int(train_set.N / batch_size)

            for i in range(total_batch):
                batch_x, batch_y = train_set.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
                avg_cost += c / total_batch

            if epoch % display_step == 0:
                print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))
            
            epoch += 1

            curr = avg_cost
        print("Optimization Finished!")

        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print("Accuracy:", accuracy.eval({x: test_set.images, y: test_set.labels}))

run_session()

Initialized
Epoch: 0001 cost= 5317722.406547851
Epoch: 0002 cost= 938564.356705729
Epoch: 0003 cost= 482634.179828695
Epoch: 0004 cost= 269126.894649455
Epoch: 0005 cost= 161492.367887471
Epoch: 0006 cost= 101838.613998718
Epoch: 0007 cost= 64983.281977742
Epoch: 0008 cost= 45125.435084254
Epoch: 0009 cost= 36969.837975388
Epoch: 0010 cost= 30374.841157583
Epoch: 0011 cost= 22548.229545403
Epoch: 0012 cost= 18064.333616333
Epoch: 0013 cost= 13628.150514743
Epoch: 0014 cost= 14108.167677538
Epoch: 0015 cost= 10509.112812581
Epoch: 0016 cost= 8503.398343970
Epoch: 0017 cost= 9120.725404053
Epoch: 0018 cost= 6726.712616043
Epoch: 0019 cost= 5394.512922770
Epoch: 0020 cost= 4163.577182148
Epoch: 0021 cost= 3769.025327199
Epoch: 0022 cost= 2398.687618027
Epoch: 0023 cost= 2296.365816288
Epoch: 0024 cost= 2713.713639374
Epoch: 0025 cost= 1810.510803935
Epoch: 0026 cost= 2437.068922908
Epoch: 0027 cost= 926.899783827
Epoch: 0028 cost= 1370.730035960
Epoch: 0029 cost= 1330.727856849
Epoch: 003